# Lab 9 Inverse RL & Reward Inference  
### Adversarial Inverse Reinforcement Learning (AIRL) on **LunarLander-v2**

In this lab you will:

1. Train or load an **expert** policy with PPO (forward RL).
2. Collect **demonstrations** from the expert.
3. Train **AIRL** to infer a reward and learn a policy from demos.

Run on Google Colab:

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/duke-trust-lab/intro_modern_rl/blob/main/lab9/lab9.ipynb)

---

## Learning objectives

By the end of this notebook, you should be able to:

- Explain the difference between **RL**, **IRL**, and **imitation learning**.
- Describe (at a high level) how **AIRL** uses an adversarial game to learn a reward.
- Run an AIRL pipeline end-to-end and evaluate whether the learned policy matches expert behavior.


### Concept refresher

**Forward RL:** reward is given, learn a policy.

- Policy:  π(a|s)
- Objective: maximize expected return  
  **J(π) = E[∑ γᵗ r(sₜ, aₜ)]**

**Inverse RL:** demonstrations are given, infer a reward (and usually a policy).

- Given expert trajectories τᴱ = (s₀,a₀,s₁,a₁,…)
- Find r̂(s,a) such that the optimal (or near-optimal) policy under r̂ explains the demonstrations.

**AIRL (Adversarial IRL):** trains a discriminator to distinguish expert transitions from policy transitions.
The discriminator structure corresponds to a learned reward + shaping term, encouraging a *reward that generalizes* beyond the expert’s exact state distribution.



In [ ]:
!pip -q uninstall -y gymnasium shimmy stable-baselines3 imitation box2d-py Box2D pygame

!pip -q install -U pip setuptools wheel
!apt-get -qq update
!apt-get -qq install -y swig ffmpeg

!pip -q install "gymnasium==0.29.1" "shimmy==1.3.0" "stable-baselines3==2.3.2" "imitation==1.0.0" "imageio[ffmpeg]"

!pip -q install "Box2D" "pygame==2.5.2" --only-binary=:all:


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package swig4.0.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting up swig (4.0.2

In [ ]:
import os, numpy as np, torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import gymnasium as gym

from stable_baselines3 import PPO
from imitation.util.util import make_vec_env
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet

OUT_DIR = "outputs_lab11"
os.makedirs(OUT_DIR, exist_ok=True)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

We’ll use:
- `evaluate_policy`: compute mean/std episodic return for a policy
- `record_video`: record an MP4 for qualitative inspection

> Qualitative inspection matters in IRL because “high reward” is not the only thing we care about ... we care about **matching behavior**


In [ ]:
def evaluate_policy(model, env_id="LunarLander-v2", n_episodes=10, deterministic=True, max_steps=1000, seed=0):
    rewards = []
    for ep in range(n_episodes):
        env = gym.make(env_id)
        obs, _ = env.reset(seed=seed + ep)
        total = 0.0
        for _ in range(max_steps):
            action, _ = model.predict(obs, deterministic=deterministic)
            obs, r, terminated, truncated, _ = env.step(action)
            total += float(r)
            if terminated or truncated:
                break
        rewards.append(total)
        env.close()
    return float(np.mean(rewards)), float(np.std(rewards))


def record_video(model, env_id="LunarLander-v2", out_dir=OUT_DIR, name="policy_demo", fps=30, max_steps=1000):
    env = gym.make(env_id, render_mode="rgb_array")
    frames = []
    obs, _ = env.reset()
    total = 0.0
    for _ in range(max_steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, r, terminated, truncated, _ = env.step(action)
        total += float(r)
        frames.append(env.render())
        if terminated or truncated:
            break
    env.close()
    path = os.path.join(out_dir, f"{name}.mp4")
    imageio.mimsave(path, frames, fps=fps, codec="libx264")
    print(f"Saved: {path} | Return: {total:.1f} | Steps: {len(frames)}")
    return path, total


### Train or load an Expert (Forward RL)

We first train an expert PPO policy on the environment’s **true reward**.  
Then AIRL will try to reproduce expert behavior **without directly using the reward** (only demos)


In [ ]:
ENV_ID = "LunarLander-v2"
EXPERT_PATH = os.path.join(OUT_DIR, "expert_ppo.zip")

# PPO params

expert_cfg = dict(
    n_envs=8, #<--adjust as needed (16)
    total_timesteps=500_000, #<--adjust as needed (200_000)
    learning_rate=3e-4,
    n_steps=1024, #<--adjust as needed (2048)
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
)


In [ ]:
def load_or_train_expert(env_id=ENV_ID, expert_path=EXPERT_PATH, seed=42, verbose=1, **cfg):
    rng = np.random.default_rng(seed=seed)
    env = make_vec_env(
        env_id,
        n_envs=cfg["n_envs"],
        rng=rng,
        post_wrappers=[lambda env, idx: RolloutInfoWrapper(env)]
    )

    if os.path.exists(expert_path):
        print("Loading existing expert:", expert_path)
        expert = PPO.load(expert_path, env=env)
    else:
        print("Training expert PPO")
        expert = PPO(
            "MlpPolicy",
            env,
            verbose=verbose,
            learning_rate=cfg["learning_rate"],
            n_steps=cfg["n_steps"],
            batch_size=cfg["batch_size"],
            n_epochs=cfg["n_epochs"],
            gamma=cfg["gamma"],
            gae_lambda=cfg["gae_lambda"],
            clip_range=cfg["clip_range"],
            ent_coef=cfg["ent_coef"],
            seed=seed,
        )
        expert.learn(total_timesteps=cfg["total_timesteps"])
        expert.save(expert_path)
        print("Saved expert to:", expert_path)

    mean_r, std_r = evaluate_policy(expert, env_id=env_id, n_episodes=10)
    print(f"Expert eval: {mean_r:.1f} ± {std_r:.1f} (10 eps)")

    return expert, env, mean_r, std_r

expert, vec_env, expert_mean, expert_std = load_or_train_expert(**expert_cfg)

Loading existing expert: outputs_lab11/expert_ppo.zip
Expert eval: 247.2 ± 19.5 (10 eps)


**Quick check:** record one expert episode for qualitative sanity.

If the expert is good, you’ll see controlled hovering and a safe landing (most of the time).


In [ ]:
_ = record_video(expert, ENV_ID, OUT_DIR, name="expert_demo")


Saved: outputs_lab11/expert_demo.mp4 | Return: 254.8 | Steps: 438


---

### Collect expert demonstrations

AIRL trains using a dataset of transitions sampled from expert rollouts.  
We’ll collect `min_timesteps` transitions across multiple trajectories.

> For IRL, the **coverage** of demonstrations matters: if the expert never visits some region of the state space, the inferred reward may be unreliable there.


In [ ]:
def collect_demonstrations(expert, env, min_timesteps=20_000, seed=42):
    rng = np.random.default_rng(seed=seed)
    rollouts_ = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=min_timesteps),
        rng=rng
    )
    transitions_ = rollout.flatten_trajectories(rollouts_)
    print(f"Collected transitions: {len(transitions_)}")
    print(f"Number of trajectories: {len(rollouts_)}")
    return transitions_, rollouts_

MIN_DEMO_TIMESTEPS = 50_000

transitions, rollouts_raw = collect_demonstrations(expert, vec_env, min_timesteps=MIN_DEMO_TIMESTEPS)


Collected transitions: 52187
Number of trajectories: 164


### Train AIRL

AIRL components:

- **Generator / policy** (PPO): produces rollouts to improve.
- **Discriminator**: tries to tell expert transitions apart from generator transitions.
- **Reward network**: inferred reward used by the generator policy.

In this notebook we use a shaped reward network:
`BasicShapedRewardNet = r_θ(s,a) + γΦ(s') − Φ(s)`

That shaping term helps represent *equivalent rewards* that produce the same optimal behavior


In [ ]:
def train_airl(
    transitions,
    env_id=ENV_ID,
    out_dir=OUT_DIR,
    load_existing=True,
    verbose=1,
    # env
    n_envs=8,
    env_seed=999,
    # reward net
    use_next_state=False,
    use_done=True,
    reward_hid_sizes=(256, 256),
    potential_hid_sizes=(256, 256),
    discount_factor=0.99,
    # generator PPO
    gen_learning_rate=3e-4,
    gen_n_steps=1024,
    gen_batch_size=64,
    gen_n_epochs=10,
    gen_gamma=0.99,
    gen_gae_lambda=0.95,
    gen_clip_range=0.2,
    gen_ent_coef=0.01,
    # discriminator
    disc_learning_rate=3e-4,
    n_disc_updates_per_round=4,
    demo_batch_size=512,
    # training
    gen_train_timesteps_multiplier=1.0,
    total_timesteps=300_000,
):
    # AIRL training environment
    env_airl = make_vec_env(
        env_id,
        n_envs=n_envs,
        rng=np.random.default_rng(seed=env_seed),
        post_wrappers=[lambda env, idx: RolloutInfoWrapper(env)]
    )

    reward_net = BasicShapedRewardNet(
        env_airl.observation_space,
        env_airl.action_space,
        use_state=True,
        use_action=True,
        use_next_state=use_next_state,
        use_done=use_done,
        reward_hid_sizes=list(reward_hid_sizes),
        potential_hid_sizes=list(potential_hid_sizes),
        discount_factor=discount_factor,
    )

    # gen_train_timesteps is how many env steps PPO runs per AIRL round
    gen_train_timesteps = int(n_envs * gen_n_steps * gen_train_timesteps_multiplier)

    learner = PPO(
        "MlpPolicy",
        env_airl,
        verbose=verbose,
        learning_rate=gen_learning_rate,
        n_steps=gen_n_steps,
        batch_size=gen_batch_size,
        n_epochs=gen_n_epochs,
        gamma=gen_gamma,
        gae_lambda=gen_gae_lambda,
        clip_range=gen_clip_range,
        ent_coef=gen_ent_coef,
    )

    policy_path = os.path.join(out_dir, "airl_policy.zip")
    reward_path = os.path.join(out_dir, "airl_reward_net.pt")

    if load_existing and os.path.exists(policy_path) and os.path.exists(reward_path):
        print("Loading existing AIRL artifacts")
        learner = PPO.load(policy_path, env=env_airl)
        reward_net.load_state_dict(torch.load(reward_path, map_location="cpu"))

    airl = AIRL(
        demonstrations=transitions,
        demo_batch_size=demo_batch_size,
        venv=env_airl,
        gen_algo=learner,
        reward_net=reward_net,
        allow_variable_horizon=True,
        disc_opt_kwargs={"lr": disc_learning_rate},
        n_disc_updates_per_round=n_disc_updates_per_round,
        gen_train_timesteps=gen_train_timesteps,
    )

    if not (load_existing and os.path.exists(policy_path) and os.path.exists(reward_path)):
        print(f"Training AIRL for {total_timesteps:,} timesteps")
        airl.train(total_timesteps=total_timesteps)
        print("Saving AIRL artifacts")
        airl.gen_algo.save(policy_path)
        torch.save(airl._reward_net.state_dict(), reward_path)

    mean_r, std_r = evaluate_policy(airl.gen_algo, env_id=env_id, n_episodes=20)
    print(f"AIRL eval: {mean_r:.1f} ± {std_r:.1f} (20 eps)")

    return airl, mean_r, std_r


In [ ]:
airl_cfg_fast = dict(
    n_envs=4,
    reward_hid_sizes=(256, 256),
    potential_hid_sizes=(256, 256),
    demo_batch_size=512,
    disc_learning_rate=3e-4,
    n_disc_updates_per_round=4,
    gen_learning_rate=3e-4,
    gen_n_steps=1024,
    gen_gae_lambda=0.95,
    gen_ent_coef=0.01,
    gen_train_timesteps_multiplier=1.0,
    total_timesteps=300_000,
    load_existing=True,
    verbose=1
)

airl_cfg = dict(
    n_envs=8,
    reward_hid_sizes=(512, 512),
    potential_hid_sizes=(512, 512),
    demo_batch_size=1024,
    disc_learning_rate=4.9e-4,
    n_disc_updates_per_round=4,
    gen_learning_rate=6.35e-4,
    gen_n_steps=2048,
    gen_gae_lambda=0.949,
    gen_ent_coef=0.0488,
    gen_train_timesteps_multiplier=0.813,
    total_timesteps=800_000, #<- May want to reduce, takes ~20 mins to run on an A100 with 800k timesteps
    load_existing=True,
    verbose=1
)

airl, airl_mean, airl_std = train_airl(transitions, **airl_cfg)


Using cuda device
Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.
Training AIRL for 800,000 timesteps


round:   0%|          | 0/60 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 91.3     |
|    gen/rollout/ep_rew_mean  | -174     |
|    gen/time/fps             | 2388     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 0.485    |
|    disc/disc_loss                   | 0.924    |
|    disc/disc_proportion_expert_pred | 1        |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   2%|▏         | 1/60 [00:19<19:10, 19.50s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 83.1        |
|    gen/rollout/ep_rew_mean         | -204        |
|    gen/rollout/ep_rew_wrapped_mean | 1.7         |
|    gen/time/fps                    | 2375        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 32768       |
|    gen/train/approx_kl             | 0.019998265 |
|    gen/train/clip_fraction         | 0.296       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.37       |
|    gen/train/explained_variance    | 0.14343667  |
|    gen/train/learning_rate         | 0.000635    |
|    gen/train/loss                  | -0.0726     |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.0214     |
|    gen/train/value_loss            | 0.021  

...

round:  98%|█████████▊| 59/60 [18:48<00:19, 19.29s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 962         |
|    gen/rollout/ep_rew_mean         | -0.916      |
|    gen/rollout/ep_rew_wrapped_mean | -1.1e+03    |
|    gen/time/fps                    | 2435        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 983040      |
|    gen/train/approx_kl             | 0.009657002 |
|    gen/train/clip_fraction         | 0.135       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.951      |
|    gen/train/explained_variance    | 0.89644456  |
|    gen/train/learning_rate         | 0.000635    |
|    gen/train/loss                  | 4.73        |
|    gen/train/n_updates             | 590         |
|    gen/train/policy_gradient_loss  | -0.0152     |
|    gen/train/value_loss            | 10.3   

round: 100%|██████████| 60/60 [19:07<00:00, 19.13s/it]

Saving AIRL artifacts


AIRL eval: -30.1 ± 23.4 (20 eps)


### Compare performance

We’ll compare:
- Expert mean return
- AIRL mean return

### Checkpoint
If AIRL gets *high reward* but looks behaviorally different, is that “success”? Why or why not?


In [ ]:
print(f"Expert: {expert_mean:.1f} ± {expert_std:.1f}")
print(f"AIRL:   {airl_mean:.1f} ± {airl_std:.1f}")


Expert: 247.2 ± 19.5
AIRL:   -30.1 ± 23.4


Record an AIRL episode.  
Compare the video with the expert: smoothness, hovering, leg contact timing, crash vs safe landings, etc.


In [ ]:
_ = record_video(airl.gen_algo, ENV_ID, OUT_DIR, name="airl_demo")


Saved: outputs_lab11/airl_demo.mp4 | Return: -3.2 | Steps: 1000


### Checkpoint

*Reflect on the following statements about IRL. How do your results highlight some of these challenges with IRL?*

IRL ≠ imitation → AIRL struggles because it must infer why actions were taken

High-quality demos are necessary but not sufficient

Reward learning is fragile

Small reward errors → catastrophic policy failure, especially in long-horizon control

In [ ]:
# TODO
# How can you improve the performance of AIRL?

### Checkpoint

1. Why can multiple different rewards explain the same demonstrations?
2. If AIRL learned a reward that works near the expert’s states, why might it fail off-distribution?
3. What evaluation would you add to judge *behavioral similarity* beyond episodic return?
4. Can AIRL learn a reward that “looks right” on demos but induces unintended behavior? Give a concrete example in LunarLander


## TODO: Hyperparameter search (Optuna)


In [ ]:
# Uncomment to run a tiny Optuna search (can still take a while)
# !pip -q install optuna

# import optuna
# from optuna.pruners import MedianPruner
# from optuna.samplers import TPESampler

# def objective(trial):
#     reward_width = trial.suggest_categorical("reward_width", [128, 256, 512])
#     disc_lr = trial.suggest_float("disc_lr", 1e-5, 1e-3, log=True)
#     gen_lr = trial.suggest_float("gen_lr", 1e-5, 1e-3, log=True)
#     ent = trial.suggest_float("ent_coef", 0.001, 0.05, log=True)
#     n_envs = trial.suggest_categorical("n_envs", [2, 4, 8])

#     airl_tmp, mean_r, std_r = train_airl(
#         transitions,
#         n_envs=n_envs,
#         reward_hid_sizes=(reward_width, reward_width),
#         potential_hid_sizes=(reward_width, reward_width),
#         disc_learning_rate=disc_lr,
#         gen_learning_rate=gen_lr,
#         gen_ent_coef=ent,
#         total_timesteps=120_000,
#         load_existing=False,
#         verbose=0
#     )
#     return mean_r

# study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=0), pruner=MedianPruner())
# study.optimize(objective, n_trials=10)

# print("Best trial:", study.best_trial.number)
# print("Best value:", study.best_value)
# print("Best params:", study.best_params)


### Checkpoint

- What are we optimizing in AIRL (at a high level)?
- What does it mean for IRL to “succeed” beyond having a high episodic return?

